In [4]:
import os
import sys

# Set the directory where 'src' is located
project_root = os.getcwd()  # Or replace this with the path to your project root
src_directory = os.path.join(project_root, 'src')

# Add 'src' directory to sys.path
sys.path.append(src_directory)

import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch import nn
from torchvision import models, transforms

# Now these imports should work
from src.utils import download_images
from src.constants import entity_unit_map

import os
import tempfile
from PIL import Image
from src.utils import download_images

from sklearn.preprocessing import LabelEncoder

class ProductImageDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, transform=None, limit=None):
        self.data = pd.read_csv(csv_file)
        if limit:
            self.data = self.data.head(limit)  # Limit to the first `limit` rows
        self.transform = transform
        self.temp_dir = tempfile.mkdtemp()

        # Initialize LabelEncoder to convert string labels to numerical labels
        if 'entity_value' in self.data.columns:
            self.label_encoder = LabelEncoder()
            self.data['entity_value'] = self.label_encoder.fit_transform(self.data['entity_value'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_url = self.data.loc[idx, 'image_link']
        download_images([img_url], self.temp_dir)
        filename = os.path.basename(img_url)
        image_path = os.path.join(self.temp_dir, filename)
        image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
        if self.transform:
            image = self.transform(image)

        if 'entity_value' in self.data.columns:
            label = torch.tensor(self.data.loc[idx, 'entity_value'], dtype=torch.long)
            return image, label
        
        return image, self.data.loc[idx, 'index']
# Define the model
class EntityExtractionModel(nn.Module):
    def __init__(self, num_classes):
        super(EntityExtractionModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)
# Main training function
def train_model(train_csv, val_csv, num_epochs=10, train_limit=200):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Limit training dataset to 200 samples
    train_dataset = ProductImageDataset(train_csv, transform=transform, limit=train_limit)
    val_dataset = ProductImageDataset(val_csv, transform=transform)

    num_classes = train_dataset.data['entity_value'].nunique()
    model = EntityExtractionModel(num_classes=num_classes)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # The rest of your training loop remains the same
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            labels = labels.long()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                labels = labels.long()  # Ensure labels are of type long
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Train Loss: {loss.item():.4f}, '
              f'Val Loss: {val_loss/len(val_loader):.4f}, '
              f'Val Accuracy: {100 * correct / total:.2f}%')

    return model

# Function to generate predictions
def generate_predictions(model, test_csv, output_csv, num_test_cases=50):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    test_dataset = ProductImageDataset(test_csv, transform=transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    predictions = []
    model.eval()
    with torch.no_grad():
        count = 0
        for inputs, indices in test_loader:
            if count >= num_test_cases:
                break
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            for idx, pred in zip(indices, predicted):
                if count >= num_test_cases:
                    break
                predictions.append([idx, f"{pred.item()}"])
                count += 1

    pd.DataFrame(predictions, columns=['index', 'prediction']).to_csv(output_csv, index=False)

if __name__ == "__main__":
    train_csv = os.path.join('dataset', 'train.csv')
    val_csv = os.path.join('dataset', 'test.csv')
    test_csv = os.path.join('dataset', 'sample_test.csv')
    output_csv = os.path.join('dataset', 'sample_test_out.csv')

    # Train the model using only 200 training samples
    model = train_model(train_csv, val_csv, num_epochs=10, train_limit=200)
    generate_predictions(model, test_csv, output_csv, num_test_cases=50)

    from src.sanity import check_output_format
    check_output_format(output_csv)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


KeyboardInterrupt: 

In [1]:
import os
import sys

# Set the directory where 'src' is located
project_root = os.getcwd()  # Or replace this with the path to your project root
src_directory = os.path.join(project_root, 'src')

# Add 'src' directory to sys.path
sys.path.append(src_directory)

import pandas as pd
import numpy as np
from PIL import Image
import torch
from torch import nn
from torchvision import models, transforms

# Now these imports should work
from src.utils import download_images
from src.constants import entity_unit_map

import os
import tempfile
from PIL import Image
from src.utils import download_images

from sklearn.preprocessing import LabelEncoder

class ProductImageDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, num_rows=None, transform=None):
        # Load only a subset of rows from the CSV file
        self.data = pd.read_csv(csv_file).head(num_rows) if num_rows else pd.read_csv(csv_file)
        self.transform = transform
        self.temp_dir = tempfile.mkdtemp()

        # Initialize LabelEncoder to convert string labels to numerical labels
        if 'entity_value' in self.data.columns:
            self.label_encoder = LabelEncoder()
            self.data['entity_value'] = self.label_encoder.fit_transform(self.data['entity_value'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_url = self.data.loc[idx, 'image_link']
        download_images([img_url], self.temp_dir)
        filename = os.path.basename(img_url)
        image_path = os.path.join(self.temp_dir, filename)
        image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
        if self.transform:
            image = self.transform(image)

        if 'entity_value' in self.data.columns:
            label = torch.tensor(self.data.loc[idx, 'entity_value'], dtype=torch.long)  # Ensure label is a tensor and long type
            return image, label
        
        return image, self.data.loc[idx, 'index']

# Define the model
class EntityExtractionModel(nn.Module):
    def __init__(self, num_classes):
        super(EntityExtractionModel, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Main training function
def train_model(train_csv, val_csv, num_epochs=10):
    # Your existing transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load only 200 rows for training and 50 rows for validation
    train_dataset = ProductImageDataset(train_csv, num_rows=200, transform=transform)
    val_dataset = ProductImageDataset(val_csv, num_rows=50, transform=transform)

    num_classes = train_dataset.data['entity_value'].nunique()  # Dynamically set num_classes
    model = EntityExtractionModel(num_classes=num_classes)  # Update the model to use the correct num_classes

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # The rest of your training loop remains the same...

    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            print(f'Inputs shape: {inputs.shape}, Labels shape: {labels.shape}')  # Debug line
            optimizer.zero_grad()
            outputs = model(inputs)
            labels = labels.long()  # Ensure labels are of type long
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                labels = labels.long()  # Ensure labels are of type long
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Train Loss: {loss.item():.4f}, '
              f'Val Loss: {val_loss/len(val_loader):.4f}, '
              f'Val Accuracy: {100 * correct / total:.2f}%')

    return model

# Function to generate predictions
def generate_predictions(model, test_csv, output_csv):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    test_dataset = ProductImageDataset(test_csv, num_rows=50, transform=transform)  # Only load 50 rows for testing
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    predictions = []
    model.eval()
    with torch.no_grad():
        for inputs, indices in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            for idx, pred in zip(indices, predicted):
                predictions.append([idx, f"{pred.item()} {entity_unit_map[pred.item()]}"])

    pd.DataFrame(predictions, columns=['index', 'prediction']).to_csv(output_csv, index=False)

# Main execution
if __name__ == "__main__":
    # Update these paths to use the correct directory structure
    train_csv = os.path.join('dataset', 'train.csv')
    val_csv = os.path.join('dataset', 'test.csv')  # You might need to create this from train.csv
    test_csv = os.path.join('dataset', 'sample_test.csv')
    output_csv = os.path.join('dataset', 'sample_test_out.csv')

    model = train_model(train_csv, val_csv)
    generate_predictions(model, test_csv, output_csv)

    # Run sanity check
    from src.sanity import check_output_format
    check_output_format(output_csv)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1/1 [00:00<00:00,  2.32it/s]


Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32])


100%|██████████| 1/1 [00:00<00:00,  2.03it/s]


Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32])


100%|██████████| 1/1 [00:00<00:00,  1.96it/s]


...

100%|██████████| 1/1 [00:00<00:00,  1.62it/s]


Inputs shape: torch.Size([32, 3, 224, 224]), Labels shape: torch.Size([32])


100%|██████████| 1/1 [00:01<00:00,  1.37s/it]


Inputs shape: torch.Size([8, 3, 224, 224]), Labels shape: torch.Size([8])


100%|██████████| 1/1 [00:00<00:00,  2.13it/s]
